Verify identical embeddings from local Chroma DB vs. Elastic Search

In [3]:
import RAG_utils

import pandas as pd
import tqdm
%load_ext autoreload
%autoreload 2

In [4]:
# db = RAG_utils.create_chroma_db(doc_path='documents/train_size_1000_overlap_200_documents.pkl', collection_name='train_full')
db = RAG_utils.access_chroma_db(collection_name='train_1000_200_all-MiniLM-L6-v2')
db._collection.count()

45877

In [5]:
import json
f = open('context.json',) 
aws_embeddings = json.load(f)

In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
def compare_lists_rounded(list1, list2, precision):
    rounded_list1 = [round(x, precision) if isinstance(x, float) else x for x in list1]
    rounded_list2 = [round(x, precision) if isinstance(x, float) else x for x in list2]
    return rounded_list1 == rounded_list2

In [9]:
precision = 4
chroma_docs = []
for i, doc in enumerate(aws_embeddings):
    
    doc_id = doc['_source']['doc_id']
    vector = doc['_source']['vector']
    aws_string = doc['_source']['sentence']
    query = aws_string
    vector = model.embed_query(query)
    x = db._collection.query(vector, n_results=1, include=['documents', 'embeddings'])
    chroma_string = x['documents'][0][0]
    chroma_embedding = x['embeddings'][0][0]
    #print(len(chroma_embedding))
    chroma_docs.append(x)

    # make sure retrieved doc is same doc from es
    assert aws_string == chroma_string, f"===AWS===\n {aws_string}\n\n ===Chroma=== \n{chroma_string}"
    # check if the embeddings are the same
    assert compare_lists_rounded(chroma_embedding, vector, precision), f"{i}"
 
    

In [10]:
x['documents'][0]

['- 24337 W Boulevard De John3 beds, 2 baths\n1,734 sqft, 22,651 sqft lot, built in 1974\n- 3304 Mistflower Ln4 beds, 5 baths\n3,114 sqft, 10,123 sqft lot, built in 2003\nRentals\n- 24330 W Boulevard De John3 beds, 2 baths\n-- sqft, -- sqft lot, built in --\n- 3412 Minito Ct5 beds, 5 baths\n4,950 sqft, -- sqft lot, built in 2005\n- 3007 Saganashkee Ln2 beds, 3 baths\n1,850 sqft, -- sqft lot, built in 2003\n- 3007 Saganashkee Ln # 30072 beds, 2.5 baths\n1,850 sqft, -- sqft lot, built in 2003\n- 3443 Breitwieser Ln4 beds, 3.5 baths\n3,118 sqft, 10,100 sqft lot, built in 2001\n- 2809 Wheatland Ct2 beds, 2.5 baths\n1,550 sqft, -- sqft lot, built in 1998\n- 2817 Ledgestone Ct3 beds, 2.5 baths\n1,500 sqft, -- sqft lot, built in 1998\n- 2812 Cantore Rd3 beds, 2.5 baths\n1,502 sqft, -- sqft lot, built in 1997\n- 2876 Cedar Glade Dr3 beds, 2.5 baths\n1,624 sqft, -- sqft lot, built in 1997\n- 11336 Highland Dr S4 beds, 2.5 baths\n2,200 sqft, -- sqft lot, built in 2005\n- 11338 Highland Dr S4 bed

In [11]:
db._collection.get(include=['documents','embeddings'], limit=1)

{'ids': ['00005183-118e-409b-9c2c-6809ea50daf9'],
 'embeddings': [[0.010060473345220089,
   0.13437420129776,
   -0.007699828594923019,
   0.0002206151548307389,
   0.02392825298011303,
   0.032303836196660995,
   0.023796483874320984,
   0.09846016764640808,
   0.0031379021238535643,
   0.06893547624349594,
   -0.03573177382349968,
   0.010137896053493023,
   -0.07930430769920349,
   0.055071886628866196,
   -0.12286555767059326,
   -0.08197297155857086,
   0.04640600085258484,
   -0.07548345625400543,
   -0.0719911977648735,
   0.028220094740390778,
   0.018637213855981827,
   -0.0124208340421319,
   0.05536636337637901,
   0.013548324815928936,
   -0.09411904960870743,
   -0.06323109567165375,
   0.028568604961037636,
   -0.09701638668775558,
   0.06912533938884735,
   0.010881921276450157,
   0.07776914536952972,
   0.0011258022859692574,
   0.013437829911708832,
   -0.03424461558461189,
   0.02298874594271183,
   -0.005880038253962994,
   -0.03360600396990776,
   0.007449227850884

In [13]:
question = 'When was the house at 3524 Redwing Ct, Naperville, IL 60564 last sold and for what price?'
documents = db.similarity_search_with_relevance_scores(question, k=5, )
for i, doc in enumerate(documents):
    print(f'Document {i+1} ------------------------------------')
    print(doc[1])
    print(doc[0].page_content)
    print('\n\n')
    

ValueError: You must provide an embedding function to compute embeddings.https://docs.trychroma.com/embeddings